# Synapse Analysis Training in Colab

This notebook provides a simple workflow for training models from the synapse analysis project in Google Colab.

## 1. Setup Environment

In [ ]:
# Install required packages
!pip install transformers scikit-learn matplotlib seaborn torch torchvision umap-learn openpyxl

In [ ]:
# Clone the repository
!git clone https://github.com/alim98/testsyn.git
%cd testsyn

## 2. Download and Prepare Data

If you've uploaded data files to Colab's file system, you can skip this section. Otherwise, download the data from Google Drive:

In [ ]:
# Mount Google Drive (if your data is there)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Download data directly from shared links if needed
!wget -O downloaded_file.zip "https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000"
!wget -O vesicle_cloud__syn_interface__mitochondria_annotation.zip "https://drive.usercontent.google.com/download?id=1qRibZL3kr7MQJQRgDFRquHMQlIGCN4XP&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000"

# Unzip the data
!unzip -q downloaded_file.zip -d /content/raw
!unzip -q vesicle_cloud__syn_interface__mitochondria_annotation.zip -d /content/seg

## 3. Add Project Root to Python Path

This ensures all imports work correctly:

In [ ]:
import sys
import os

# Add the current directory to the Python path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Create necessary directories if they don't exist
os.makedirs('results', exist_ok=True)
os.makedirs('output', exist_ok=True)

## 4. Extract Point Clouds

Run the point cloud extraction script first:

In [ ]:
# Run point cloud extraction
!python extract_point_clouds.py --raw_base_dir "/content/raw" --seg_base_dir "/content/seg" --add_mask_base_dir "/content" --output_path "point_clouds.pkl"

## 5. Train the Model

Now run the training script:

In [ ]:
# Import training function
from train_with_point_clouds import train

# Set up training arguments
class Args:
    def __init__(self):
        self.point_clouds_path = "point_clouds.pkl"
        self.output_dir = "output"
        self.batch_size = 16  # Adjust based on GPU memory
        self.num_workers = 2
        self.epochs = 50
        self.learning_rate = 1e-4
        self.weight_decay = 1e-4
        self.encoder_dim = 512
        self.projection_dim = 128
        self.temperature = 0.1
        self.use_dual_encoder = True  # Set to True to use both texture and shape encoders
        self.max_points = 1024  # Maximum number of points in point cloud

args = Args()

# Run training
train(args)

## 6. Evaluate and Visualize Results

After training, you can evaluate your model and visualize the results:

In [ ]:
import torch
import matplotlib.pyplot as plt
from models.contrastive_model import DualEncoderModel
import numpy as np

# Load your trained model
model_path = os.path.join(args.output_dir, "model_final.pth")
model = DualEncoderModel(
    in_channels=1,
    encoder_dim=args.encoder_dim,
    projection_dim=args.projection_dim,
    max_points=args.max_points
)
model.load_state_dict(torch.load(model_path))
model.eval()

# Visualize the loss curve
loss_path = os.path.join(args.output_dir, "losses.npy")
if os.path.exists(loss_path):
    losses = np.load(loss_path)
    plt.figure(figsize=(10, 5))
    plt.plot(losses)
    plt.title("Training Loss")
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.show()

## 7. Save and Download Results

Download the trained model and results to your local computer:

In [ ]:
# Zip the output directory
!zip -r trained_model.zip output/

# Download the zip file (will prompt in Colab)
from google.colab import files
files.download('trained_model.zip')